In [1]:
import pandas as pd
import numpy as np
import os
import sqlite3
from datetime import datetime

import openai

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')

database = Chroma(persist_directory='./database/', embedding_function=embeddings)

In [3]:
contents = pd.read_csv('aivleschool_contents.csv')
contents.head()

,date,contents
0,2024-06-10 19:49:10,"보도자료 바로가기 \n\nKT, 에이블스쿨 6기 교육생 모집…한국 AI 경쟁력 강화..."
1,2024-06-10 19:45:03,"보도자료 바로가기 \n\nKT, 디지털인재 키우는 '에이블스쿨' 모집…""수료 시 K..."
2,2024-05-28 08:30:52,"보도자료 바로가기 \n\nKT AIVLE School, '경력 같은 신입' 양성으로..."
3,2024-05-28 08:22:00,"보도자료 바로가기 \n\nKT AIVLE School, '경력 같은 신입' 양성으로..."
4,2023-12-11 09:52:11,보도자료\n\nKT가 키운 에이블스쿨 디지털 인재 “해커톤 우승 비결은 실무형 교육...


In [4]:
text_list = contents['contents'].to_list()
metadata = []
cat_list = contents['date'].to_list()
for cat in cat_list:
    temp = {}
    temp['date'] = cat
    metadata.append(temp)
    
documents = [Document(page_content = text_list[i], metadata = metadata[i]) for i in range(len(text_list))]

print(documents)

database.add_documents(documents)

[Document(page_content='보도자료 바로가기 \n\nKT, 에이블스쿨 6기 교육생 모집…한국 AI 경쟁력 강화 앞장\n\n34세 이하 취준생 참여 가능…수료 후 KT 채용지원시 우대\n350여 개 기업 취업 연계 성사… 디지털전환 주역으로 활약\n\n\n\nKT가 청년 디지털 인재 교육을 통해 한국 AI 경쟁력 강화와 디지털 인재 양성에 앞장선다.\n KT는 다음 달 1일까지 에이블스쿨 6기 교육생을 모집한다고 10일 밝혔다. 이번 모집에서 선발된 입교생들은 9월 초부터 약 6개월간의 교육 과정을 이수한다. \n 에이블스쿨은 KT가 정부와 함께 기업 실무형 디지털 인재를 키우고 AI, 디지털 분야 인재를 필요로 하는 기업의 일자리와 연결해 국가 디지털 경쟁력 제고에 이바지하는 프로그램이다.\n 에이블스쿨 교육 과정 수료 후 KT 채용에 지원 시 서류 전형 면제와 우수 수료자 추가 우대 혜택이 제공된다.\n 에이블스쿨은 34세 이하 미취업자 가운데 4년제 대학 졸업자 혹은 2025년 2월 졸업 예정자라면 누구나 지원 가능하다. 특히 이번 6기 모집에서는 다른 SW 인재양성 프로그램 수료자를 우대하며, 코딩테스트 시 Java언어 사용자에게 가점이 주어진다.\n 교육생들은 교육 기간 총 840시간의 이론 및 실습 교육을 이수하며 기업의 실전형 프로젝트 수행 경험도 쌓는다. 특히 KT는 교육생들이 전국 어디서나 동일한 환경과 조건으로 우수한 품질의 교육을 받을 수 있도록 전용 온라인 교육과 실습 플랫폼 에이블에듀(AIVLE-EDU)를 활용 중이다.\n 이 밖에도 서울과 분당 등의 수도권을 비롯한 대전, 광주, 대구, 부산 등에 오프라인 교육장을 마련해 운영 중이며 지역 소재 교육생들로부터 호평을 받고 있다. KT의 현직 전문가들이 직접 참여해 학습 방법 외에도 실무 현장에서 일하는 방식까지 직접 교육한다. \n 교육생에게는 KT가 개발한 인공지능 자격증 AICE는 물론 빅데이터분석기사, SQLD 등 관련 분야 자격증 취득 기회도 적극적으로 제공한다. 

['0e513f11-221f-4ea5-bbd0-6636f501aa30',
 '7745bb2a-70bd-430f-b397-9b9a61ece84c',
 '56e17a04-96e6-4565-b0fe-b4a390ada8c6',
 'dc895dfa-adf4-4818-acda-01c30945a286',
 '05caf7a1-9468-4d4e-98f3-37b306dc8d9d',
 '8ae19f19-6c24-4860-b210-16dff8836183',
 '3158ca10-46f0-4770-8962-06797bf04684',
 'dc657af5-ec64-45eb-a81b-bb57233a72dd',
 'b6c86cc1-2611-45c8-9478-f2f06c7989f4',
 '3ab355ed-cc54-4239-abfd-721d0b8cfef0',
 'cf37ba13-6396-48fd-8e2f-b9533b4c8a84',
 'e1763481-fe01-4a71-a63a-f09ab50de6fa',
 '04220a02-b277-4890-b0ba-f57a980b5d4c',
 '5d5b5c5b-8b1b-4bf2-b6ba-a8122b295419',
 'ce25f50e-2a23-4799-acc0-8eb47afcd657',
 'f4ac49dd-9b2b-46f5-9a15-342bfc3fca21',
 '9c71eaf4-a115-43c6-ae4d-17663e993236',
 '24932123-a55c-4c80-8a67-f77420d5228a',
 '041c6173-d384-4a77-abd0-e4c5514be021',
 '84275cda-5216-4fd3-8c2b-e018db0be927',
 '4006bea6-a5c2-47c1-81b7-338ced5fbe0e',
 '321eec25-3727-4784-a8fe-c878cd5b9415',
 'e95bd1e1-4dcc-4757-8121-d625829c7229',
 'b86b1f63-8a3c-4cd0-aab9-4a6883201b3d',
 '8e2bb597-c2a1-

In [5]:
review = pd.read_csv('aivleschool_review.csv')
review.head()

,구분,QA
0,후기,“직무의 전환점이 되어준 에이블스쿨””1. 자기소개 부탁드립니다..안녕하세요! 에이...
1,후기,“스펙(SPEC)이 되어준 에이블스쿨”(SPEC)”1. 자기소개 부탁드립니다..안녕...
2,후기,“삼성SDS 취업 성공기”SDS ”1. 자기소개 부탁드립니다..안녕하세요. 에이블스...
3,후기,“다양한 시각으로 문제를 바라보며 생각하는 능력을 키워준 에이블스쿨””1. 자기소개...
4,후기,“IT 컨설턴트로의 성공적인 전환을 도와준 에이블스쿨””1. 자기소개 부탁드립니다....


In [6]:
text_list = review['QA'].to_list()
metadata = []
cat_list = review['구분'].to_list()
for cat in cat_list:
    temp = {}
    temp['구분'] = cat
    metadata.append(temp)
    
documents = [Document(page_content = text_list[i], metadata = metadata[i]) for i in range(len(text_list))]

print(documents)

database.add_documents(documents)

[Document(page_content='“직무의 전환점이 되어준 에이블스쿨””1. 자기소개 부탁드립니다..안녕하세요! 에이블스쿨 AI트랙 3기 수료생 이동비입니다. 학부 때 정보보안을 전공하여 AI/Bigdata 관련 직무 취업이 목표로 하였으나 높은 진입장벽에 포기하려던 중 AI 인재를 양성하는 교육프로그램인 에이블스쿨을 알게 되어 지원하게 되었습니다. 수료 후에는 에이블스쿨에서 쌓은 역량을 잘 활용하여 채용 연계를 통해 ktcs에 입사하게 되었습니다. !  AI3.  AI/Bigdata AI . ktcs. 2. ktcs에서 어떤 일을 하시나요??데이터 엔지니어로서 AICC개발팀에서 일하고 있습니다. AICC. 데이터 엔지니어는 내-외부 원천 데이터를 수집, 가공, 적재하여 데이터가 흐르는 파이프라인을 설계 및 구축하는 일을 합니다. 생성되는 전화/채팅 상담 데이터를 수집, 가공, 적재하고, NLP엔진을 활용하여 분석한 다음 대시보드에 표출하는 업무를 맡고 있습니다. 저의 업무는 주로 Python을 통한 데이터 처리에 대해 배운 부분을 많이 활용하고 있습니다. 그리고 고객사 대상 VOC분석 보고서 작성하거나, 통계분석 및 워드클라우드 제작하는 등의 업무를 수행하는데, 에이블스쿨의 기본과정으로 배우는 SQL, Linux, Python과 같은 IT공통 기초 부분들이 도움이 되고 있습니다. 때에 따라 상담데이터 분석 대시보드 제작 시, 외부 NLP엔진 API를 통한 텍스트 분석을 진행하는 과정에서 머신러닝, NLP엔진 등을 활용하는 경우가 많기 때문에 그에 대한 이해도가 많은 도움이 되고 있습니다.-, , . /, , , NLP. 저의 업무는 주로 Python을 통한 데이터 처리에 대해 배운 부분을 많이 활용하고 있습니다.Python.  VOC, ,  SQL, Linux, Python IT. ,  NLP API, NLP.3. AI개발자 과정을 통해 진로 변화가 어떻게 되었나요??전공이 데이터 분석이다 보니 입교 당시에는 데이터 분석가를 목표로 가지고 있었지만,

['a7f58794-fcd7-45a2-9074-e1544432cd2f',
 '58ec2099-2f61-4e51-b478-89bf1deda25d',
 'a517ce21-4166-44ae-af00-dd5e6b60443f',
 'b7b12d76-b9f6-434b-9977-402fc4c1c4d3',
 'dcc6b468-436d-480c-a170-2f80876f82a0',
 '213d6ec3-46ab-4e9e-9dff-d9d2dc43e4cc',
 'd67ca68d-9c29-473b-a5dd-9c5ecce8b4d4',
 'db14fd4b-4400-4cef-b005-67950f37693a',
 '52de195c-b446-4b24-9f16-073e99cc1c62',
 '28861436-7208-4103-8d34-3b8302b6e750',
 'cee47ed8-f248-4af9-9ca9-83a0f61162ef',
 '10ee9972-d8a7-43bb-9b84-a104d324d2b6',
 '85b893ef-008a-4855-a679-2d43c3deb96d',
 '65ebb035-18f1-4791-8319-bbf203bd101a',
 '5b31a0fb-8cd9-404c-ae00-de560dae7982',
 '3e3d61d8-d018-48d1-a4f1-e1955d418e6c',
 '7de3a253-3e26-496a-bde5-8beff42a04b4',
 'd1e72ac6-18b1-4ad6-94c7-2e41fbbf55e0',
 'ac6d14ac-ac1d-4c6b-9094-728311a08c8d',
 'f8b47e43-a194-4ce1-8f7c-cc53d9f2edde',
 'dfa4fc78-c765-4c11-8675-b9a91994bd1a',
 'dec23a04-b40a-45e6-9e08-84bc5f869f16',
 'a8e0ee83-8a16-4a32-9fb1-7e3366a06496',
 '032c3660-0ad8-4f78-8169-0ae7308e53ef',
 'cc59f114-89d8-

In [7]:
data = database.get()
data = pd.DataFrame(data)
data

,ids,embeddings,metadatas,documents,uris,data
0,000d4c23-a907-4baf-8018-0530b44f36fc,None,{'구분': '채용연계'},채용 연계 관련 어떤 지원이 있나요? /nKT 에이블스쿨은 미래인재육성 + 청년 ...,None,None
1,02e7e8c1-288a-40ef-a156-59bb8a5f45c7,None,{'date': '2022-09-22 09:00:00'},"보도자료 바로가기▶\n\nKT, 청년 AI 인재양성 프로그램 가동\nKT도 청년 A...",None,None
2,032c3660-0ad8-4f78-8169-0ae7308e53ef,None,{'구분': '후기'},“서로 의지하며 부족한 것을 채워준 DX ...,None,None
3,041c6173-d384-4a77-abd0-e4c5514be021,None,{'date': '2023-04-12 17:12:45'},"보도자료 바로가기 ▶\nKT, 에이블스쿨 통해 '산업 현장에서 쓰일 디지털 스킬' ...",None,None
4,04220a02-b277-4890-b0ba-f57a980b5d4c,None,{'date': '2023-07-17 09:52:51'},"보도자료 바로가기 ▶\n\nKT, '에이블스쿨' 잡페어 개최…AI역량으로 누적취업률...",None,None
...,...,...,...,...,...,...
223,f8b47e43-a194-4ce1-8f7c-cc53d9f2edde,None,{'구분': '후기'},"“에이블스쿨의 경험, 성장을 위한 첫걸음”에이블스쿨의 경험, 성장을 위한 첫걸음””...",None,None
224,f9b0f957-40bb-49a9-9fde-2ef6bedc7a96,None,{'구분': '국민내일배움카드'},"국민내일배움카드가 있어야만 지원이 가능한가요?/n네, 국민내일배움카드 보유자만 지...",None,None
225,fac5fbef-8d5a-4bc9-92a3-1f8456c90570,None,{'구분': '후기'},▶실전 경험으로 쌓은 디지털 역량 무기로취업난에도 누적 취업률 70% 웃돌아[이데...,None,None
226,fc1ad9ed-6158-4b09-9c41-360ce0be816e,None,{'date': '2022-04-10 09:00:01'},보도자료 바로가기▶\nKT가 청년 AI인재양성 프로그램 KT 에이블스쿨(KT AIV...,None,None
